In [14]:
# Disable warnings
import warnings
warnings.filterwarnings('ignore')

# Parallelization
import ipyparallel

# Time profiling
import cProfile
from   timeit import default_timer

# Combinatorics tool
import itertools

# Usual library
import numpy             as np
import pandas            as pd
import matplotlib        as mpl
import matplotlib.pyplot as plt
%matplotlib inline

# Plot settings
mpl.rcParams['legend.frameon' ] = False
mpl.rcParams['legend.fontsize'] = 'xx-large'
mpl.rcParams['xtick.labelsize'] = 16
mpl.rcParams['ytick.labelsize'] = 16
mpl.rcParams['axes.titlesize' ] = 18
mpl.rcParams['axes.labelsize' ] = 18
mpl.rcParams['lines.linewidth'] = 2.5

In [15]:
cluster = ipyparallel.Client(profile='default')
print('profile:', cluster.profile)
print("IDs:", cluster.ids) # Print process id numbers

('profile:', u'default')
('IDs:', [0, 1, 2, 3])


In [16]:
dview = cluster[:]
print(dview)

<DirectView [0, 1, 2, 3]>


In [24]:
def test_function(a):
    x=a[0]
    y=a[1]
    return x**2+(y**2+1)*x*y

arg = [ [x,y] for (x,y) in itertools.combinations(range(5000),2) ]
print( len(arg) )

12497500


In [26]:
t_start = default_timer()
serial_result   = list(map(test_function, arg ) )
print(len(serial_result))
t_stop  = default_timer()
del serial_result
print (' --> done in {:.2f}s \n'.format( t_stop-t_start ) )

12497500
 --> done in 3.67s 



In [27]:
t_start = default_timer()
parallel_result = dview.map_sync(test_function, arg)
t_stop  = default_timer()
del parallel_result
print (' --> done in {:.2f}s \n'.format( t_stop-t_start ) )

 --> done in 45.23s 



In [ ]:
# root_numpy (http://scikit-hep.org/root_numpy)
from   root_numpy import root2array
import pandautils as     pup

# Define some classes
class Jet(object):
    """
    Jet Class
    """
    def __init__(self, pt = 0 , eta = 0, phi = 0, m = 0):
        super(Jet, self).__init__()
        
        self._pt = pt
        self._eta = eta
        self._phi = phi
        self._m = m
        
    def __str__(self):
        return 'jet(pT,eta,phi,m)=({:.2f},{:.2f},{:.2f},{:.2f})'.format(self._pt, self._eta, self._phi, self._m)
        
    @property
    def pt(self):
        return self._pt
        
    @property
    def eta(self):
        return self._eta

    @property
    def phi(self):
        return self._phi
        
    @property
    def m(self):
        return self._m
    
    
class Muon(object):
    """
    Muon Class
    """
    def __init__(self, pt = 0 , eta = 0, phi = 0):
        super(Muon, self).__init__()
        self._pt = pt
        self._eta = eta
        self._phi = phi
        
    def __str__(self):
        return 'mu(pT,eta,phi)=({:.2f},{:.2f},{:.2f})'.format(self._pt, self._eta, self._phi)
        
    @property
    def pt(self):
        return self._pt
        
    @property
    def eta(self):
        return self._eta

    @property
    def phi(self):
        return self._phi

    
class Electron(object):
    """
    Electron Class
    """
    def __init__(self, pt = 0 , eta = 0, phi = 0):
        super(Electron, self).__init__()
        self._pt = pt
        self._eta = eta
        self._phi = phi
        
    def __str__(self):
        return 'el(pT,eta,phi)=({:.2f},{:.2f},{:.2f})'.format(self._pt, self._eta, self._phi)
        
    @property
    def pt(self):
        return self._pt
        
    @property
    def eta(self):
        return self._eta

    @property
    def phi(self):
        return self._phi

In [ ]:
data = pd.DataFrame( root2array('VectorNtuple_4topSM.root', 'nominal_Loose').view(np.recarray) )

In [ ]:
def add_objectCollection(obj,kinem_name,collection_name):
    obj_array = [ obj(pt, eta, phi) for (pt, eta, phi) in zip(
        pup.flatten(data[kinem_name[0]]),
        pup.flatten(data[kinem_name[1]]),
        pup.flatten(data[kinem_name[2]])
    )]
    objects = np.array( pup.match_shape( np.array(obj_array) , data[kinem_name[0]] ) )
    data[collection_name] = objects
    return

t_start = default_timer()
add_objectCollection(Jet     ,['jet_pt','jet_eta','jet_phi'],'jetCollection' )
add_objectCollection(Muon    ,['mu_pt' ,'mu_eta' ,'mu_phi' ],'muonCollection')
add_objectCollection(Electron,['el_pt' ,'el_eta' ,'el_phi' ],'electronCollection')

In [ ]:
def GetdR(j1,j2):
    return np.sqrt( np.square(j1.eta-j2.eta)+ np.square(j1.phi-j2.phi) )

def GetmindR_class(coll1,coll2):
    if (np.array_equal(coll1,coll2)): 
        all_pairs = list(itertools.combinations(coll1,2))
    else:
        all_pairs = list( itertools.product(coll1,coll2) )
    if(len(all_pairs)>0):
        return np.min([GetdR(o1,o2) for (o1,o2) in all_pairs] )
    else:
        return -1

def compute_manydR_class(c):
    dRmj=GetmindR_class(c['muonCollection']     , c['jetCollection'])
    dRjj=GetmindR_class(c['jetCollection']      , c['jetCollection'])
    dRej=GetmindR_class(c['electronCollection'] , c['jetCollection'])
    dRee=GetmindR_class(c['electronCollection'] , c['electronCollection'])
    return dRmj,dRjj,dRej,dRee

In [ ]:
t_start = default_timer()
test = data.apply( compute_manydR_class, axis=1 )
t_stop  = default_timer()
print (' --> done in {:.2f}s \n'.format( t_stop-t_start ) )